In [2]:
import pandas as pd
import torch
import copy

from sklearn.model_selection import train_test_split
from itertools import product
from sklearn.metrics import r2_score
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
#import dataset
df = pd.read_csv('dataset-ml-25m/dataset.csv')

In [5]:
# Dividi il dataset in feature e target
X = df.drop(['rating'], axis=1).to_numpy()
y = df['rating'].to_numpy()

# Dividi il dataset in training, validation e test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42)

y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)
#count the numebr of x_train 
print("Number of train set: ", X_train.shape[0])
print("Numebr of test set: ", X_test.shape[0])
print("Number of validation set: ", X_val.shape[0])


Number of train set:  9946
Numebr of test set:  2764
Number of validation set:  1106


In [6]:
batch_sizes = [8, 16, 32]
n_epochs = [200]
#dimensionality of the output space of the feature transformer network
n_d = [8, 16, 32]
 #dimensionality of the output space of the attention network
n_a = [8, 16, 32]
#number of sequential steps in the attention mechanism 
n_steps = [3, 5, 7]
params = list(product(batch_sizes, n_epochs, n_d, n_a, n_steps))


In [7]:
for b, n_e, n_d, n_a, n_s in params:
    print(f"Configuration batch size: {b} - epochs: {n_e} - n_d: {n_d} - n_a: {n_a} - steps: {n_s} ")
    model = TabNetRegressor(
        n_a = n_a,
        n_d = n_d,
        n_steps= n_s
    )

    model.fit(
        X_train=X_train,
        y_train=y_train,
        eval_set=[(X_val, y_val)],
        eval_name=['mse'],
        batch_size=b
    )
      # evaluate model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print('MSE:', mse)
    print('R2 Score:', r2)
    

Configuration batch size: 8 - epochs: 200 - n_d: 8 - n_a: 8 - steps: 3 


/usr/local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.2812  | mse_mse: 0.07043 |  0:00:31s
epoch 1  | loss: 0.09894 | mse_mse: 0.27305 |  0:01:05s
epoch 2  | loss: 0.0819  | mse_mse: 0.07393 |  0:01:37s
epoch 3  | loss: 0.07828 | mse_mse: 0.06278 |  0:02:08s
epoch 4  | loss: 0.07167 | mse_mse: 0.05142 |  0:02:38s
epoch 5  | loss: 0.07409 | mse_mse: 0.06293 |  0:03:09s
epoch 6  | loss: 0.07251 | mse_mse: 0.06103 |  0:03:39s
epoch 7  | loss: 0.06916 | mse_mse: 0.0606  |  0:04:11s
epoch 8  | loss: 0.06491 | mse_mse: 0.0412  |  0:04:41s
epoch 9  | loss: 0.0633  | mse_mse: 0.05938 |  0:05:13s
epoch 10 | loss: 0.06093 | mse_mse: 0.10407 |  0:05:42s
epoch 11 | loss: 0.06202 | mse_mse: 0.06393 |  0:06:12s
epoch 12 | loss: 0.0585  | mse_mse: 0.07598 |  0:06:41s
epoch 13 | loss: 0.05858 | mse_mse: 0.07784 |  0:07:03s
epoch 14 | loss: 0.06011 | mse_mse: 0.04215 |  0:07:24s
epoch 15 | loss: 0.05708 | mse_mse: 0.04663 |  0:07:48s
epoch 16 | loss: 0.05288 | mse_mse: 0.0658  |  0:08:11s
epoch 17 | loss: 0.04876 | mse_mse: 0.05388 |  0

/usr/local/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


MSE: 0.03906676625632957
R2 Score: 0.8237584937093532
Configuration batch size: 8 - epochs: 200 - n_d: 8 - n_a: 8 - steps: 5 


...

MSE: 0.009663572196417787
R2 Score: 0.9564048247845661
Configuration batch size: 32 - epochs: 200 - n_d: 32 - n_a: 32 - steps: 7 


/usr/local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.947   | mse_mse: 0.61709 |  0:00:15s
epoch 1  | loss: 0.16289 | mse_mse: 0.15501 |  0:00:31s
epoch 2  | loss: 0.12447 | mse_mse: 0.07389 |  0:00:48s
epoch 3  | loss: 0.09397 | mse_mse: 0.06228 |  0:01:05s
epoch 4  | loss: 0.0752  | mse_mse: 0.0422  |  0:01:22s
epoch 5  | loss: 0.07073 | mse_mse: 0.06951 |  0:01:39s
epoch 6  | loss: 0.07103 | mse_mse: 0.04148 |  0:01:55s
epoch 7  | loss: 0.05323 | mse_mse: 0.04399 |  0:02:12s
epoch 8  | loss: 0.05151 | mse_mse: 0.03945 |  0:02:29s
epoch 9  | loss: 0.04488 | mse_mse: 0.024   |  0:02:46s
epoch 10 | loss: 0.03314 | mse_mse: 0.02087 |  0:03:03s
epoch 11 | loss: 0.03776 | mse_mse: 0.04919 |  0:03:20s
epoch 12 | loss: 0.03407 | mse_mse: 0.01274 |  0:03:37s
epoch 13 | loss: 0.03489 | mse_mse: 0.04426 |  0:03:54s
epoch 14 | loss: 0.03635 | mse_mse: 0.0225  |  0:04:11s
epoch 15 | loss: 0.03413 | mse_mse: 0.02547 |  0:04:28s
epoch 16 | loss: 0.03653 | mse_mse: 0.04732 |  0:04:45s
epoch 17 | loss: 0.03363 | mse_mse: 0.01634 |  0

/usr/local/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


MSE: 0.014322879234792673
R2 Score: 0.9353853402097261
